

###  <font color='brown'> BUT DU JEU : </font>

Le jeu des bâtonnets est un jeu de duel qui demande logique et stratégie. Notre objectif c'est de mettre la machine capable de découvrir la stratégie de ce jeu.

Les joueurs retirent chacun leur tour 1, 2, ou 3 bâtonnets il ne faut pas être celui qui retirera le dernier.

In [ ]:
from random import randint
import random
import numpy as np

class StickGame(object):

    def __init__(self, nb):
        super(StickGame, self).__init__()
        self.original_nb = nb
        self.nb = nb  

    def is_finished(self):
        if self.nb <= 0:
            return True
        return False

    def reset(self):
        self.nb = self.original_nb
        return self.nb

    def display(self):
        print ("| " * self.nb,f'    ({self.nb})')

    def step(self, action):
        self.nb -= action 
        if self.nb <= 0:  
            return None, -1
        else:
            return self.nb, 0

class StickPlayer(object):
    def __init__(self, is_human, size, trainable=True):
        super(StickPlayer, self).__init__()
        self.is_human = is_human 
        self.history = [] 
        self.V = {}
        for s in range(1, size+1):
            self.V[s] = 0. 
        self.win_nb = 0.    
        self.lose_nb = 0.  
        self.rewards = []
        self.eps = 0.99    
        self.trainable = trainable

    def reset_stat(self):
        # Reset stat
        self.win_nb = 0
        self.lose_nb = 0
        self.rewards = []

    def greedy_step(self, state):
        actions = [1, 2, 3] #Le principe du jeu c'est qu'on ne peut choisir qu'au plus 3 battonets par iteration
        vmin = None
        vi = None
        for i in range(0, 3):
            a = actions[i] 
            #On va vérifier pour chaque action l'état dans lequel notre ennemi va se retrouver.
            #L'objectif c'est de choisir l'action qui mène notre ennemi à l'etat le plus défavorable (celui qui a le plus petit score)
            if state - a > 0 and (vmin is None or vmin > self.V[state - a]): 
                #Au départ tous les scores sont nulles donc on ne sait pas quel action à choisir
                vmin = self.V[state - a]
                vi = i       
        return actions[vi if vi is not None else 1]

    def play(self, state):
        if self.is_human is False:
            if random.uniform(0, 1) < self.eps: 
                #Choisir une action aléatoire (Exploration) 
                action = randint(1, 3)
            else: #Choisir une acton selon la stratégie choisit auparavant (Exploitation)
                action = self.greedy_step(state)
        else: 
            action = int(input("$>"))
            while action not in [1,2,3]:
                action = int(input("$>"))
        return action

    def add_transition(self, n_tuple):
        self.history.append(n_tuple)
        s, a, r, sp = n_tuple
        self.rewards.append(r)

    def train(self):
        if not self.trainable or self.is_human is True:
            return

        for transition in reversed(self.history): 
            
            s, a, r, sp = transition
            # s: l'etat avant de prendre l'action 
            # sp: l'etat avant de jouer la prochaine itération
            # a: l'action choisit
            # r: la récompense
            
            #Rétropagation de la récompense et mise à jour de létat qui nous mène à obtenir cette récompense
            #puis de tous les états choisis auparavant 
            if r != 0:
                self.V[s] = self.V[s] + 0.001*(r - self.V[s])
            else:
                self.V[s] = self.V[s] + 0.001*(self.V[sp] - self.V[s])

        self.history = []

def play(game, p1, p2, train=True):
    state = game.reset()
    players = [p1, p2]
    if p2.is_human:
        players = [p2, p1]
    elif not(p2.is_human)and not(p1.is_human):
        random.shuffle(players)
    p = 0
    while game.is_finished() is False: 

        if players[p%2].is_human:
            game.display()

        action = players[p%2].play(state) 
        n_state, reward = game.step(action) 
        if (reward != 0): 

            players[p%2].lose_nb += 1. if reward == -1 else 0
            players[p%2].win_nb += 1. if reward == 1 else 0

            players[(p+1)%2].lose_nb += 1. if reward == 1 else 0
            players[(p+1)%2].win_nb += 1. if reward == -1 else 0

        if p != 0:
            s, a, r, sp = players[(p+1)%2].history[-1]
            players[(p+1)%2].history[-1] = (s, a, reward * -1, n_state)

        players[p%2].add_transition((state, action, reward, None))

        state = n_state
        p += 1

    if train:
        p1.train()
        p2.train()

if __name__ == '__main__':
    game = StickGame(20) 

    p1 = StickPlayer(is_human=False, size=20, trainable=True)
    p2 = StickPlayer(is_human=False, size=20, trainable=True)

    human = StickPlayer(is_human=True, size=20, trainable=False)
    random_player = StickPlayer(is_human=False, size=20, trainable=False)

    # Train the agent
    for i in range(0, 10000): 
        if i % 10 == 0:
            p1.eps = max(p1.eps*0.996, 0.01)#En progressant dans l'apprentissage on favorise l'exploitation que de l'exploration
            p2.eps = max(p2.eps*0.996, 0.01)
        play(game, p1, p2)
    p1.reset_stat()

    # Jouer contre un joueur aléatoire
    for _ in range(0, 1000):
        play(game, p1, random_player, train=False)
    print("p1 win rate", p1.win_nb/(p1.win_nb + p1.lose_nb),'\n')

    while True:
        play(game, p1, human, train=False)
        if p1.history[-1][-2]==1: 
            print('Vous avez perdu! :p \n')
        else:
            print('Vous avez gagnez! :D \n')

p1 win rate 0.99 

| | | | | | | | | | | | | | | | | | | |      (20)
$>2
| | | | | | | | | | | | | | | | |      (17)
$>2
| | | | | | | | | | | | |      (13)
$>2
| | | | | | | | |      (9)
$>1
| | | | |      (5)
$>2
|      (1)
$>1
Vous avez perdu! :p 

| | | | | | | | | | | | | | | | | | | |      (20)


In [ ]:
print('Q_value:')
for key in p1.V:
    print('---------------------------')
    print('|',key,'|',p1.V[key])
print("---------------------------")

###  <font color='brown'> Stratégie : </font>

Le but est de toujours laisser l'autre condidat avec : 17, 13, 9, 5 puis 1 batonnet pour gagner la partie.
On remarque que la machine est capable de découvrir cette stratégie puisque tous les états mentionnés auparavant ont des scores négatifs donc à chaque itération la machine tente à laisser l'autre candidat avec 17, 13, 9, 5 puis 1 battonet pour gagner. 